In [1]:
import os
import re
import json
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from keras.utils import array_to_img, load_img
from keras.layers import TextVectorization
from keras.applications.resnet import ResNet152, preprocess_input
from keras.utils import Progbar
import matplotlib.pyplot as plt
from mrcnn.utils import extract_bboxes

In [2]:
seed = int(time.time())
np.random.seed(seed)
tf.random.set_seed(seed)

CAPTIONS_PATH = "./annotations/captions_train2014.json"
IMAGES_PATH = "./train2014/train2014"
IMAGE_SIZE = (224, 224)
VOCAB_SIZE = 30000
SEQ_LENGTH=50
AUTOTUNE = tf.data.AUTOTUNE

In [3]:
def load_captions_data(filename):
    """Loads captions (text) data and maps them to corresponding images.

    Args:
        filename: Path to the text file containing caption data.

    Returns:
        caption_mapping: Dictionary mapping image names and the corresponding captions
        text_data: List containing all the available captions
    """
    with open(filename) as f:
        json_data = json.load(f)
        imageList = json_data["images"]
        captions = json_data["annotations"]

        images = {}
        # Index all images
        for image in imageList:
            imageId, imageName = image["id"], image["file_name"]
            images[imageId] = os.path.join(IMAGES_PATH, imageName)

        caption_mapping = {}
        text_data = []
        images_to_skip = set()

        for captionDict in captions:
            if captionDict["image_id"] not in images:
                continue

            img_name = images[captionDict["image_id"]]
            caption = captionDict["caption"]

            # We will remove caption that are either too short to too long
            tokens = caption.strip().split()

            if len(tokens) < 5 or len(tokens) > SEQ_LENGTH:
                images_to_skip.add(img_name)
                continue

            if img_name.endswith("jpg") and img_name not in images_to_skip:
                # We will add a start and an end token to each caption
                caption = "<start> " + caption.strip() + " <end>"
                text_data.append(caption)

                if img_name in caption_mapping:
                    if len(caption_mapping[img_name]) < 5:
                        caption_mapping[img_name].append(caption)
                else:
                    caption_mapping[img_name] = [caption]

        for img_name in images_to_skip:
            if img_name in caption_mapping:
                del caption_mapping[img_name]

        return caption_mapping, text_data


def train_val_split(caption_data, train_size=0.8, shuffle=True):
    """Split the captioning dataset into train and validation sets.

    Args:
        caption_data (dict): Dictionary containing the mapped caption data
        train_size (float): Fraction of all the full dataset to use as training data
        shuffle (bool): Whether to shuffle the dataset before splitting

    Returns:
        Traning and validation datasets as two separated dicts
    """

    # 1. Get the list of all image names
    all_images = list(caption_data.keys())

    # 2. Shuffle if necessary
    if shuffle:
        np.random.shuffle(all_images)

    # 3. Split into training and validation sets
    train_size = int(len(caption_data) * train_size)

    training_data = {
        img_name: caption_data[img_name] for img_name in all_images[:train_size]
    }
    validation_data = {
        img_name: caption_data[img_name] for img_name in all_images[train_size:]
    }

    # 4. Return the splits
    return training_data, validation_data


# Load the dataset
captions_mapping, text_data = load_captions_data(CAPTIONS_PATH)

# Split the dataset into training and validation sets
train_data, valid_data = train_val_split(captions_mapping)
print("Number of training samples: ", len(train_data))
print("Number of validation samples: ", len(valid_data))

Number of training samples:  66226
Number of validation samples:  16557


In [4]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


strip_chars = "!\"#$%&'()*+,-./:;=?@[\]^_`{|}~"

vectorization = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=SEQ_LENGTH,
    standardize=custom_standardization,
)
vectorization.adapt(text_data)
VOCAB_SIZE = vectorization.vocabulary_size()
vocab = vectorization.get_vocabulary()

In [5]:
BATCH_SIZE=64

def decode_and_resize(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    return img


def process_input(img_path, captions):
    return decode_and_resize(img_path), vectorization(captions)


def make_dataset(images, captions):
    dataset = tf.data.Dataset.from_tensor_slices((images, captions))
    dataset = dataset.shuffle(BATCH_SIZE * 8)
    dataset = dataset.map(process_input, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return dataset


# # Pass the list of images and the list of corresponding captions
train_dataset = make_dataset(list(train_data.keys()), list(train_data.values()))
valid_dataset = make_dataset(list(valid_data.keys()), list(valid_data.values()))

In [6]:
def get_cnn_model():
    base_model = ResNet152(
        input_shape=(*IMAGE_SIZE, 3),
        include_top=False,
        weights="imagenet",
    )
    # We freeze our feature extractor
    base_model.trainable = False
    base_model_out = base_model.output
    base_model_out = layers.Reshape((-1, base_model_out.shape[-1]))(base_model_out)
    cnn_model = keras.models.Model(base_model.input, base_model_out)
    return cnn_model

def get_glove_embeddings(vocab):
    word_lookup = dict(zip(vocab, range(VOCAB_SIZE)))

    glove_path = "glove.6B/glove.6B.300d.txt"
    embeddings_index = {}
    with open(glove_path, encoding="utf8") as f:
        for line in f:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, "f", sep=" ")
            if word in word_lookup:
                embeddings_index[word] = coefs

    print("Found %s word vectors" % len(embeddings_index))
    embedding_matrix = np.zeros((VOCAB_SIZE, 300))
    for word, i in word_lookup.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

def positional_encoding(position, d_model):
    pos = np.arange(position)[:, np.newaxis]
    den = np.power(
        10000, (2 * (np.arange(d_model)[np.newaxis, :] // 2) / np.float32(d_model))
    )
    angle_rads = pos / den
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads
    return tf.cast(pos_encoding, dtype=tf.float32)


class SeqEmbedding(layers.Layer):
    def __init__(self, vocab_size, d_model, seq_length, word_embeddings):
        super().__init__()
        self.glove_embedding = layers.Embedding(
            vocab_size,
            300,
            embeddings_initializer=keras.initializers.Constant(word_embeddings),
            mask_zero=True,
        )
        self.resizer = layers.Dense(d_model, activation="relu")
        self.pos_encoding = positional_encoding(seq_length, d_model)
        self.add = layers.Add()

    def call(self, x):
        length = tf.shape(x)[1]
        x = self.glove_embedding(x)
        x = self.resizer(x)
        return self.add([x, self.pos_encoding[tf.newaxis, :length, :]])

class BaseAttention(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = layers.MultiHeadAttention(**kwargs)
        self.layernorm = layers.LayerNormalization()
        self.add = layers.Add()


# Decoder cross attention layer
class CrossAttention(BaseAttention):
    def call(self, x, context, training):
        attn_output = self.mha(query=x, key=context, value=context, training=training)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x


class GlobalSelfAttention(BaseAttention):
    def call(self, x, training):
        attn_output = self.mha(query=x, key=x, value=x, training=training)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x


class CausalSelfAttention(BaseAttention):
    def call(self, x, training):
        attn_output = self.mha(
            query=x, key=x, value=x, use_causal_mask=True, training=training
        )
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x


class FeedForward(layers.Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.seq = keras.Sequential(
            [
                layers.Dense(dff, activation="relu"),
                layers.Dense(d_model),
                layers.Dropout(dropout_rate),
            ]
        )
        self.add = layers.Add()
        self.layer_norm = layers.LayerNormalization()

    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x)
        return x


class EncoderLayer(layers.Layer):
    def __init__(self, *, d_model, num_heads, dff, dropout_rate=0.1):
        super().__init__()

        self.self_attention = GlobalSelfAttention(
            num_heads=num_heads, key_dim=d_model, dropout=dropout_rate
        )

        self.ffn = FeedForward(d_model, dff)

    def call(self, x, training):
        x = self.self_attention(x, training)
        x = self.ffn(x)
        return x


class Encoder(layers.Layer):
    def __init__(self, *, num_layers, d_model, num_heads, dff, dropout_rate=0.1):
        super().__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.layer_norm = layers.LayerNormalization()
        self.dense = layers.Dense(d_model, activation="relu")

        self.enc_layers = [
            EncoderLayer(
                d_model=d_model, num_heads=num_heads, dff=dff, dropout_rate=dropout_rate
            )
            for _ in range(num_layers)
        ]
        self.dropout = layers.Dropout(dropout_rate)

    def call(self, x, training):
        # `x` is token-IDs shape: (batch, seq_len)
        x = self.layer_norm(x)
        x = self.dense(x)

        # Add dropout.
        x = self.dropout(x)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training)

        return x  # Shape `(batch_size, seq_len, d_model)`.


class DecoderLayer(layers.Layer):
    def __init__(self, *, d_model, num_heads, dff, dropout_rate=0.1):
        super(DecoderLayer, self).__init__()

        self.causal_self_attention = CausalSelfAttention(
            num_heads=num_heads, key_dim=d_model, dropout=dropout_rate
        )

        self.cross_attention = CrossAttention(
            num_heads=num_heads, key_dim=d_model, dropout=dropout_rate
        )

        self.ffn = FeedForward(d_model, dff)

    def call(self, x, context, training):
        x = self.causal_self_attention(x, training)
        x = self.cross_attention(x, context, training)
        x = self.ffn(x)
        return x


class Decoder(layers.Layer):
    def __init__(
        self,
        *,
        num_layers,
        d_model,
        num_heads,
        dff,
        vocab_size,
        seq_length,
        word_embeddings,
        dropout_rate=0.1
    ):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.pos_embedding = SeqEmbedding(
            vocab_size=vocab_size,
            d_model=d_model,
            seq_length=seq_length,
            word_embeddings=word_embeddings,
        )
        self.norm = layers.LayerNormalization()
        self.dropout = layers.Dropout(dropout_rate)
        self.dec_layers = [
            DecoderLayer(
                d_model=d_model, num_heads=num_heads, dff=dff, dropout_rate=dropout_rate
            )
            for _ in range(num_layers)
        ]

        self.out = layers.Dense(VOCAB_SIZE, activation="softmax")

    def call(self, x, context, training):
        # `x` is token-IDs shape (batch, target_seq_len)
        x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)
        x = self.norm(x)
        x = self.dropout(x)

        for i in range(self.num_layers):
            x = self.dec_layers[i](x, context, training)

        # self.last_attn_scores = self.dec_layers[-1].last_attn_scores
        x = self.out(x)

        # The shape of x is (batch_size, target_seq_len, d_model).
        return x

In [7]:
class DenseCaptioner(keras.Model):
    def __init__(
        self,
        cnn_model,
        encoder,
        decoder,
        num_captions_per_image=5,
        image_aug=None,
    ):
        super().__init__()
        self.cnn_model = cnn_model
        self.encoder = encoder
        self.decoder = decoder
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.acc_tracker = keras.metrics.Mean(name="accuracy")
        self.num_captions_per_image = num_captions_per_image
        self.image_aug = image_aug

    def calculate_loss(self, y_true, y_pred):
        loss = self.loss(y_true, y_pred)
        mask = (y_true != 0) & (loss < 1e8)
        mask = tf.cast(mask, loss.dtype)
        loss *= mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)

    def calculate_accuracy(self, y_true, y_pred):
        mask = y_true != 0
        accuracy = tf.equal(y_true, tf.argmax(y_pred, axis=2))
        accuracy = tf.math.logical_and(mask, accuracy)
        accuracy = tf.cast(accuracy, dtype=tf.float32)
        mask = tf.cast(mask, tf.float32)
        return tf.reduce_sum(accuracy) / tf.reduce_sum(mask)

    def _compute_caption_loss_and_acc(self, img_embed, batch_seq, training=True):
        encoder_out = self.encoder(img_embed, training=training)
        batch_seq_inp = batch_seq[:, :-1]
        batch_seq_true = batch_seq[:, 1:]
        batch_seq_pred = self.decoder(batch_seq_inp, encoder_out, training=training)
        loss = self.calculate_loss(batch_seq_true, batch_seq_pred)
        acc = self.calculate_accuracy(batch_seq_true, batch_seq_pred)
        return loss, acc

    def train_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0

        img_embed = self.cnn_model(preprocess_input(batch_img))

        for i in range(self.num_captions_per_image):
            with tf.GradientTape() as tape:
                loss, acc = self._compute_caption_loss_and_acc(
                    img_embed, batch_seq[:, i, :], training=True
                )
                batch_loss += loss
                batch_acc += acc

            train_vars = (
                self.encoder.trainable_variables + self.decoder.trainable_variables
            )
            grads = tape.gradient(loss, train_vars)
            self.optimizer.apply_gradients(zip(grads, train_vars))

        batch_acc /= float(self.num_captions_per_image)
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        # 8. Return the loss and accuracy values
        return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}

    def test_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0

        img_embed = self.cnn_model(preprocess_input(batch_img))

        for i in range(self.num_captions_per_image):
            loss, acc = self._compute_caption_loss_and_acc(
                img_embed, batch_seq[:, i, :], training=False
            )

            # 3. Update batch loss and batch accuracy
            batch_loss += loss
            batch_acc += acc

        batch_acc /= float(self.num_captions_per_image)

        # 4. Update the trackers
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        # 5. Return the loss and accuracy values
        return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker, self.acc_tracker]

    def forward(self, img):
        img = self.cnn_model(preprocess_input(img))

        # Pass the image features to the Transformer encoder
        encoded_img = self.encoder(img, training=False)

        # Generate the caption using the Transformer decoder
        decoded_caption = "<start> "
        for i in range(max_decoded_sentence_length):
            tokenized_caption = vectorization([decoded_caption])[:, :-1]
            predictions = self.decoder(
                tokenized_caption, encoded_img, training=False
            )
            sampled_token_index = np.argmax(predictions[0, i, :])
            sampled_token = index_lookup[sampled_token_index]
            if sampled_token == "<end>":
                break
            decoded_caption += " " + sampled_token

        decoded_caption = decoded_caption.replace("<start> ", "")
        decoded_caption = decoded_caption.replace(" <end>", "").strip()
        return decoded_caption

In [8]:
EMBED_DIM = 512
FF_DIM = 512
EPOCHS = 10
NUM_HEADS = 8
ENCODER_LAYERS = 2
DECODER_LAYERS = 2

cnn_model = get_cnn_model()
word_embeddings = get_glove_embeddings(vocab)

# Init transformer blocks
encoder = Encoder(
    num_layers=ENCODER_LAYERS, d_model=EMBED_DIM, num_heads=NUM_HEADS, dff=FF_DIM
)
decoder = Decoder(
    num_layers=DECODER_LAYERS,
    d_model=EMBED_DIM,
    num_heads=NUM_HEADS,
    vocab_size=VOCAB_SIZE,
    dff=FF_DIM,
    seq_length=SEQ_LENGTH,
    word_embeddings=word_embeddings,
)

# Init dense captioner
caption_model = DenseCaptioner(
    cnn_model=cnn_model,
    encoder=encoder,
    decoder=decoder,
)

Found 19795 word vectors


In [9]:
cross_entropy = keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction="none"
)

# EarlyStopping criteria
early_stopping = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)


# Learning Rate Scheduler for the optimizer
class LRSchedule(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, post_warmup_learning_rate, warmup_steps):
        super().__init__()
        self.post_warmup_learning_rate = post_warmup_learning_rate
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        global_step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        warmup_progress = global_step / warmup_steps
        warmup_learning_rate = self.post_warmup_learning_rate * warmup_progress
        return tf.cond(
            global_step < warmup_steps,
            lambda: warmup_learning_rate,
            lambda: self.post_warmup_learning_rate,
        )


# Create a learning rate schedule
num_train_steps = len(train_dataset) * EPOCHS
num_warmup_steps = num_train_steps // 15
lr_schedule = LRSchedule(post_warmup_learning_rate=1e-4, warmup_steps=num_warmup_steps)

# Compile the model
caption_model.compile(optimizer=keras.optimizers.Adam(lr_schedule), loss=cross_entropy)

checkpoint_path = "training-glove/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
)

if os.path.exists(checkpoint_dir):
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    caption_model.load_weights(latest)


In [ ]:
caption_model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=valid_dataset,
    callbacks=[early_stopping, cp_callback],
)

Epoch 1/10
   1/1035 [..............................] - ETA: 18:24:11 - loss: 9.2075 - acc: 0.5590

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[]


In [10]:
CLASS_NAMES = [
    "BG",
    "person",
    "bicycle",
    "car",
    "motorcycle",
    "airplane",
    "bus",
    "train",
    "truck",
    "boat",
    "traffic light",
    "fire hydrant",
    "stop sign",
    "parking meter",
    "bench",
    "bird",
    "cat",
    "dog",
    "horse",
    "sheep",
    "cow",
    "elephant",
    "bear",
    "zebra",
    "giraffe",
    "backpack",
    "umbrella",
    "handbag",
    "tie",
    "suitcase",
    "frisbee",
    "skis",
    "snowboard",
    "sports ball",
    "kite",
    "baseball bat",
    "baseball glove",
    "skateboard",
    "surfboard",
    "tennis racket",
    "bottle",
    "wine glass",
    "cup",
    "fork",
    "knife",
    "spoon",
    "bowl",
    "banana",
    "apple",
    "sandwich",
    "orange",
    "broccoli",
    "carrot",
    "hot dog",
    "pizza",
    "donut",
    "cake",
    "chair",
    "couch",
    "potted plant",
    "bed",
    "dining table",
    "toilet",
    "tv",
    "laptop",
    "mouse",
    "remote",
    "keyboard",
    "cell phone",
    "microwave",
    "oven",
    "toaster",
    "sink",
    "refrigerator",
    "book",
    "clock",
    "vase",
    "scissors",
    "teddy bear",
    "hair drier",
    "toothbrush",
]

In [20]:
index_lookup = dict(zip(range(len(vocab)), vocab))
max_decoded_sentence_length = SEQ_LENGTH - 1
valid_images = list(valid_data.keys())


def generate_caption(img_id):
    sample_img = decode_and_resize(img_id)
    img = sample_img.numpy().clip(0, 255).astype(np.uint8)
    img = tf.expand_dims(sample_img, 0)
    decoded_caption = caption_model.forward(img)
    return decoded_caption

def generate_dense_caption(img_path, rcnn):
    # sample_img = np.random.choice(valid_images)
    sample_img = img_path

    Img = np.array(load_img(sample_img))
    results = rcnn.detect([Img], verbose=0)
    masks = results[0]["masks"][:, :, :8]
    # Rois = extract_bboxes(masks)
    # masks = np.transpose(masks, (2, 0, 1))
    # num_rois = masks.shape[0]
    rois = results[0]['rois']
    Rois = rois[:8]
    # print(rois)
    classes = results[0]['class_ids']
    scores = results[0]['scores']
    regions = []
    decoded_captions = []
    for j in range(len(Rois)):
        start1 = max(0, Rois[j][0] - 40)
        start2 = max(0, Rois[j][1] - 40)
        height = min(Img.shape[0] - start1, (Rois[j][2] - Rois[j][0]) * 2)
        wd = min(Img.shape[1] - start2, (Rois[j][3] - Rois[j][1]) * 2)
        masked_roi = tf.image.crop_to_bounding_box(
            Img,
            start1,
            start2,
            height,
            wd,
        )
        class_name = CLASS_NAMES[ classes[j] ]
        regions.append([class_name, masked_roi])
        masked_roi = tf.image.resize(masked_roi, IMAGE_SIZE, 'lanczos5', antialias=True)
        masked_roi = tf.expand_dims(masked_roi,0)
        decoded_captions.append(caption_model.forward(masked_roi))
    return decoded_captions, scores, regions, masks, classes[:8], Rois

In [11]:
class MyConfig(Config):
    NAME = "my_config"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 80  # COCO dataset has 80 classes + background


model_rcnn = MaskRCNN(mode="inference", model_dir=os.getcwd(), config=MyConfig())
model_rcnn.load_weights("mask_rcnn_coco.h5", by_name=True)

Instructions for updating:
box_ind is deprecated, use box_indices instead
Instructions for updating:
Use fn_output_signature instead
Instructions for updating:
Use `tf.cast` instead.


In [ ]:
img_id = "test6.jpg"
# plt.imshow(load_img(img_id))
# plt.show()
# captions = generate_caption(img_id)
captions, scores, regions = generate_dense_caption(img_id, model_rcnn)
# print(scores)
# print(captions)
for caption in captions:
    print(caption)
for i in range(len(regions)):
    print(scores[i])

for k in regions:
    print(k[0])
    plt.imshow(k[1])
    plt.show()

In [ ]:
import numpy as np
from flask import Flask, request
from tensorflow import keras
from PIL import Image
from flask import jsonify
from flask_cors import CORS
import numpy as np
import cv2
import pybase64

model = None
app = Flask(__name__)
CORS(app)

li = ['Common Bunt', 'Common Root Rot', 'Fusarium Head Blight', 'Leaf Rust', 'Powdery Mildew']

def load_model():
    global model
    model = keras.models.load_model('AlexNet_Model_Local_Dataset_Trained.h5')

@app.route('/')
def home_endpoint():
    return 'Hello World!'

def image_array_to_base64(image_array):
    # Convert image array to image object
    image = np.array(image_array, dtype=np.uint8)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    _, encoded_image = cv2.imencode(".png", image)

    # Convert image to Base64 string
    base64_string = pybase64.b64encode(encoded_image.tobytes()).decode("utf-8")

    return base64_string


@app.route('/predict', methods=['POST'])
def get_prediction():
    if request.method == 'POST':

        if 'image' not in request.files:
            return jsonify({'error': 'No file part'})
        file = request.files['image']
        img = Image.open(file.stream)
        img = img.resize((256, 256))
        img_arr = np.array(img)
        img_arr = np.expand_dims(img_arr, axis=0)
        img_arr = img_arr/255.0


        prediction = model.predict(img_arr)  
        d = prediction.flatten()
        print(d)
        j = d.max()
        print(j)
        for index,item in enumerate(d):
            if item == j:
                class_name = li[index]
    return jsonify({'prediction': str(class_name)})

if __name__ == '__main__': 
    app.run(host='0.0.0.0', port=80)

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


In [29]:
# Targets = list(valid_data.items())[:2]
Targets = ["kitchen.jpg", "kitchen2.jpg"]
from mrcnn import visualize
for img_id in Targets:
    captions, scores, regions, masks, classes, Rois = generate_dense_caption(img_id, model_rcnn)
    plt.imshow(load_img(img_id))
    plt.show()
    print(img_id)
    visualize.display_instances(
        image=np.array(load_img(img_id)),
        boxes=np.array(Rois),
        masks=np.array(masks),
        class_ids=np.array(classes),
        class_names=CLASS_NAMES,
        scores=np.array(scores[:8]),
    )
    print(captions)

FileNotFoundError: [Errno 2] No such file or directory: 'bhalu.jpg'

In [ ]:
Targets = list(valid_data.items())[:1000]

predictions = []
references = []
print(len(Targets))
for idx, (img_id, captions) in enumerate(Targets):
    predictions = generate_dense_caption(img_id, model_rcnn)
    print(idx)
    prediction = generate_caption(img_id)
    ref = [''] * 5
    for i in range(len(captions)):
        ref[i] = captions[i].split()
        ref[i] = ref[i][1 : len(ref[i]) - 1]
    references.append(ref)
    predictions.append(prediction.split())
    
print(references)

1000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

In [ ]:
from rouge import Rouge 
rouge = Rouge()

scores = 0
cnt = 0
for i in range(len(predictions)):
    prediction = ' '.join(predictions[i])
    for j in range(5):
        ref=' '.join(references[i][j])
        score = rouge.get_scores(prediction, ref)
        scores += score[0]['rouge-l']['f']
        cnt += 1
        
print(scores / cnt)

0.35216819889450796


In [ ]:
with open('preds_glove.txt', 'w') as f:
    for p in predictions:
        p = ' '.join(p)
        # print(p)
        f.write(p + '\n')

with open('refs_glove.txt', 'w') as f:
    for ref in references:
        for r in ref:
            r = ' '.join(r)
            print(r)
            f.write(r + '\n')

A couple of giraffe walking through a lush green field.
Two giraffes in a field enclosure from a distance
An enclosed area for giraffes. One area is for grazing.
Giraffes and grazing animals in one of several pens.
Two giraffes and a zebra in an outdoor zoo
A blue sign sitting in the middle of a park.
A pet rest area sign behind a fire hydrant.
A fire hydrant with a sign posted next to it that reads "Pet Rest Area".
A small fire hydrant by a sign for pet rest area.
Grassy area with a "pet rest area" sign next to a fire hydrant
A large, old-fashioned clock stand below a tree.
The clock has been erected near a large building.
an image of a clock that is going in the air
a tall pole with a clock on the top saying its 5 o clock
A clock sitting in front of a building at night.
A bird with its eyes open is sitting on top of a branch in a tree.
A brown and speckled bird on a branch.
A little, brown bird on a tree branch
Small brown bird sitting on a branch outside.
A brown bird sitting on a s